In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_im_path = 'insert your image path here'
driving_video_path = 'insert your video path here'

source_image = imageio.imread(source_im_path)
driving_video = imageio.mimread(driving_video_path)

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, orig=None, generated=None, compared=None, ep=None, clip=None, save=True):
    fig = plt.figure(figsize=(8 + 4 * (orig is not None) + 4 * (generated is not None) + 4 * (compared is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        title = 'Source image, Kon{0}_{1}'.format(ep, clip)
        if orig is not None:
            cols.append(orig[i])
            title += ', celeb chkpt'
        if generated is not None:
            cols.append(generated[i])
            title += ', kon1 chkpt'
        if compared is not None:
            cols.append(compared[i])
            title += ', kon1_5 chkpt'
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')        
        plt.title(title)
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    if save:
        ani.save('kon{0}_{1}.mp4'.format(ep, clip))
    plt.close()
    return ani

In [ ]:
from Fomm.demo import load_checkpoints
from Fomm.demo import make_animation
from skimage import img_as_ubyte

celeb_gen, celeb_kp_det = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='data/vox-cpk.pth.tar')

gen_k1, kp_det_k1 = load_checkpoints(config_path='config/kon_ep1_ray_v1.yaml', 
                            checkpoint_path='weights/kon1-checkpoint.pth.tar')

gen_k1_5, kp_det_k1_5 = load_checkpoints(config_path='config/kon_ep1_ray_v1.yaml', 
                            checkpoint_path='weights/kon1-checkpoint.pth.tar')

In [ ]:
orig_pred = make_animation(source_image, driving_video, celeb_gen, celeb_kp_det, relative=True)
pred_k1 = make_animation(source_image, driving_video, gen_k1, kp_det_k1, relative=True)
pred_k1_5 = make_animation(source_image, driving_video, gen_k1_5, kp_det_k1_5, relative=True)\

In [ ]:
HTML(display(source_image, driving_video, orig_pred, pred_k1,
             pred_k1_5).to_html5_video())